<a href="https://colab.research.google.com/github/JavierPachas/gcp_aviation/blob/main/aviationstack_extract_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install Package

!pip install google-cloud
!pip install google-cloud-pubsub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.9/265.9 kB 5.2 MB/s eta 0:00:00


In [ ]:
import os
import requests
import json
from time import sleep
from google.cloud import pubsub_v1

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/content/bigdata-project-405803-480262332163.json'
# TODO(developer)

project_id = "bigdata-project-XXXXX" #from your gcp account
# topic_id = "your-topic-id"
topic_id = "vuelos"
publisher = pubsub_v1.PublisherClient()
project_path = f"projects/{project_id}"

# AVIATION API parameters
access_key = "9c47217a00b325eXXXXXXXXXXXXXXXXX" #change it, provided by aviationstack.com API
limit = "5"
flight_status = "active"
arr_iata = "BCN"

aviation_url = "http://api.aviationstack.com/v1/flights" + \
            "?access_key=" + access_key + \
            "&flight_status=" + flight_status + \
            "&arr_iata=" + arr_iata
            # "&limit=" + limit

topic_path = publisher.topic_path(project_id, topic_id)
# Main code
while True:
    # Get all flights
    response = requests.get(aviation_url)
    if response.status_code != 200:
        print("Algo ocurrió. ERROR: ", response.status_code)
        print(response.json)
        exit()

    json_text = response.text

    python_obj = json.loads(json_text)

    data_array = []
    for flight in python_obj['data']:
        data = {}

        # Only flight in the air
        if flight['live'] != None:
            data['vuelo'] = flight['flight']['iata']
            data['aerolinea'] = flight['airline']['name']
            data['fecha_vuelo'] = flight['flight_date']
            data['origen'] = flight['departure']['airport']
            data['destino'] = flight['arrival']['airport']
            data['latitud'] = flight['live']['latitude']
            data['longitud'] = flight['live']['longitude']
            data['altitud'] = flight['live']['altitude']
            data['velocidad'] = flight['live']['speed_horizontal']
            data['timestamp'] = flight['live']['updated']
            print("Enviando información del vuelo " + data['vuelo'])
            data_array.append(data)

    print("\nEnviando array: ")
    print(data_array)

    # When you publish a message, the client returns a future.
    future = publisher.publish(topic_path, json.dumps(data_array).encode("utf-8"))
    print(future.result())

    print("\n")

    # Wait till next refresh
    sleep(5)


Enviando información del vuelo CA845
Enviando información del vuelo AC822
Enviando información del vuelo AA112
Enviando información del vuelo IB2602
Enviando información del vuelo UA992
Enviando información del vuelo DL194

Enviando array: 
[{'vuelo': 'CA845', 'aerolinea': 'Air China LTD', 'fecha_vuelo': '2023-12-07', 'origen': 'Beijing Capital International', 'destino': 'El Prat De Llobregat', 'latitud': 61.8399, 'longitud': 50.8645, 'altitud': 11582.4, 'velocidad': 814.88, 'timestamp': '2023-12-07T01:28:43+00:00'}, {'vuelo': 'AC822', 'aerolinea': 'Air Canada', 'fecha_vuelo': '2023-12-06', 'origen': 'Pierre Elliott Trudeau International', 'destino': 'El Prat De Llobregat', 'latitud': 45.7312, 'longitud': -44.9839, 'altitud': 11879.6, 'velocidad': 1150.09, 'timestamp': '2023-12-07T02:13:00+00:00'}, {'vuelo': 'AA112', 'aerolinea': 'American Airlines', 'fecha_vuelo': '2023-12-06', 'origen': 'Miami International Airport', 'destino': 'El Prat De Llobregat', 'latitud': 39.0554, 'longitud': 

KeyboardInterrupt: ignored